In [1]:
## This file is going to run models which will attempt to forecast price movement 
## using data from 2016 until Jan 31st, 2022. The first 1, 2, 5, and 10 trading days 
## of Februrary will be predicted. 

In [22]:
import pandas as pd

import glob

from pathlib import Path
import csv

import glob

import datetime

In [2]:
# Machine-learning specific imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import model_from_json

import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [3]:
## API Imports 
import os
import requests
import json

from dotenv import load_dotenv
load_dotenv()

True

In [4]:
## Load pickle for exports and imports of data  
import pickle 
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
    
def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [5]:
# ## FMP Constants 
# fmpbase_urlv3 = 'https://fmpcloud.io/api/v3/'
# fmpbase_urlv4 = 'https://fmpcloud.io/api/v4/'
# api_key = os.getenv("FMP_CLOUD_API_KEY")

# start_date = '2022-01-01'
# end_date = '2022-01-31'

# ## FMP Functions 
# def get_FMP_historical_data(symbol, startDate=start_date, endDate=end_date, apiKey=api_key):
#     url_hist_price = fmpbase_urlv3+'historical-price-full/'
#     url_hist_query_with_date = url_hist_price+symbol+'?from='+startDate+'&to='+endDate+'&apikey='+apiKey
#     resp_data = requests.get(url_hist_query_with_date)
#     json_ = resp_data.json()
#     data = json_['historical']
#     df = pd.DataFrame(data)
#     df.rename(columns={'date':'Date'},inplace=True)
#     df['Date'] = pd.to_datetime(df['Date'])
#     df = df.reindex(index=df.index[::-1]) ## Reverse the DataFrame 
#     df.set_index('Date',inplace=True)
#     df.drop(columns='label',inplace=True)
    
#     ## Export and save data so API calls aren't repeated 
#     path = Path('../FilesExport_Updated_API_data/'+symbol+'_jan_2022.pkl')
#     save_obj(df,path)
    
#     return df

In [9]:
## Load df for ML data prep 

symbol = 'AMC'

import_path = Path('../FilesExport_Updated_DFs_01_31/'+symbol+'_ti_df_no_ftd.pkl')
data = load_obj(import_path)
import_df = data[symbol]
import_df

,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,...,ShortVolumeNSDQ,ShortExemptVolumeNSDQ,TotalVolumeNSDQ,ShortVolumeNYSE,ShortExemptVolumeNYSE,TotalVolumeNYSE,bollinger_signal,dema_signal,adl_signal,rsi_signal
Date,,,,,,,,,,,,,,,,,,,,,
2016-03-02,24.940001,25.389999,24.620001,25.350000,19.225668,177300.0,177300.0,0.41,1.644,25.12000,...,24261.0,0.0,39418.0,1860.0,0.0,4100.0,0.124078,0.927201,-0.533333,0.907210
2016-03-03,25.240000,25.780001,25.129999,25.700001,19.646111,264500.0,264500.0,0.46,1.823,25.53667,...,35886.0,2336.0,69522.0,8558.0,0.0,11805.0,0.114017,0.939775,-0.633333,0.885180
2016-03-04,26.400000,27.240000,25.400000,27.020000,20.655172,1264100.0,1264100.0,0.62,2.348,26.55333,...,136925.0,1887.0,380932.0,37132.0,2700.0,146211.0,0.099262,0.947055,-0.783333,0.943925
2016-03-07,26.990000,28.740000,26.840000,28.680000,21.924139,433500.0,433500.0,1.69,6.262,28.08667,...,50718.0,800.0,129688.0,7091.0,0.0,24014.0,0.057679,0.964262,-0.883333,0.922563
2016-03-08,28.459999,28.840000,28.299999,28.389999,21.702448,486000.0,486000.0,-0.07,-0.246,28.51000,...,40113.0,0.0,106944.0,7116.0,0.0,146998.0,0.065728,0.966909,-0.933333,0.983311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-25,15.890000,16.619400,15.550000,16.020000,16.020000,42220722.0,42220722.0,0.13,0.818,16.06313,...,8158731.0,176867.0,14548715.0,2597484.0,84667.0,5789849.0,0.970490,0.058240,-0.535570,0.109479
2022-01-26,16.210000,18.155000,15.650000,15.940000,15.940000,76432209.0,76432209.0,-0.27,-1.666,16.58167,...,18838070.0,182242.0,29763994.0,5638109.0,16451.0,11711555.0,0.963783,0.070152,-0.418414,0.173565
2022-01-27,16.110000,16.580000,14.395000,14.520000,14.520000,50074448.0,50074448.0,-1.59,-9.870,15.16500,...,12476790.0,153690.0,19039258.0,3704413.0,5565.0,6834055.0,0.983233,0.061549,0.008368,0.110814


In [10]:
df - import_df.copy()

df_close = df[['close']]
df_close = df_close.reset_index().rename(columns={"Date": "Close_Date"})
df_close

,Close_Date,close
0,2016-03-02,25.350000
1,2016-03-03,25.700001
2,2016-03-04,27.020000
3,2016-03-07,28.680000
4,2016-03-08,28.389999
...,...,...
1486,2022-01-25,16.020000
1487,2022-01-26,15.940000
1488,2022-01-27,14.520000
1489,2022-01-28,15.060000


In [11]:
features_df = df.reset_index().drop(columns=['close','adjClose'])
features_df

,Date,open,high,low,volume,unadjustedVolume,change,changePercent,vwap,changeOverTime,ShortVolumeNSDQ,ShortExemptVolumeNSDQ,TotalVolumeNSDQ,ShortVolumeNYSE,ShortExemptVolumeNYSE,TotalVolumeNYSE,bollinger_signal,dema_signal,adl_signal,rsi_signal
0,2016-03-02,24.940001,25.389999,24.620001,177300.0,177300.0,0.41,1.644,25.12000,0.01644,24261.0,0.0,39418.0,1860.0,0.0,4100.0,0.124078,0.927201,-0.533333,0.907210
1,2016-03-03,25.240000,25.780001,25.129999,264500.0,264500.0,0.46,1.823,25.53667,0.01823,35886.0,2336.0,69522.0,8558.0,0.0,11805.0,0.114017,0.939775,-0.633333,0.885180
2,2016-03-04,26.400000,27.240000,25.400000,1264100.0,1264100.0,0.62,2.348,26.55333,0.02348,136925.0,1887.0,380932.0,37132.0,2700.0,146211.0,0.099262,0.947055,-0.783333,0.943925
3,2016-03-07,26.990000,28.740000,26.840000,433500.0,433500.0,1.69,6.262,28.08667,0.06262,50718.0,800.0,129688.0,7091.0,0.0,24014.0,0.057679,0.964262,-0.883333,0.922563
4,2016-03-08,28.459999,28.840000,28.299999,486000.0,486000.0,-0.07,-0.246,28.51000,-0.00246,40113.0,0.0,106944.0,7116.0,0.0,146998.0,0.065728,0.966909,-0.933333,0.983311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1486,2022-01-25,15.890000,16.619400,15.550000,42220722.0,42220722.0,0.13,0.818,16.06313,0.00818,8158731.0,176867.0,14548715.0,2597484.0,84667.0,5789849.0,0.970490,0.058240,-0.535570,0.109479
1487,2022-01-26,16.210000,18.155000,15.650000,76432209.0,76432209.0,-0.27,-1.666,16.58167,-0.01666,18838070.0,182242.0,29763994.0,5638109.0,16451.0,11711555.0,0.963783,0.070152,-0.418414,0.173565
1488,2022-01-27,16.110000,16.580000,14.395000,50074448.0,50074448.0,-1.59,-9.870,15.16500,-0.09870,12476790.0,153690.0,19039258.0,3704413.0,5565.0,6834055.0,0.983233,0.061549,0.008368,0.110814
1489,2022-01-28,14.600000,15.250000,13.400000,53945344.0,53945344.0,0.46,3.151,14.57000,0.03151,11830875.0,136735.0,19465415.0,4404829.0,3169.0,7848759.0,0.987928,0.087359,0.050000,0.211615


In [13]:
# for col in features_df.columns:
#     print(col)

In [15]:
## This cuts n_days from the top of the dataframe, and resets index

n_days = 10
new_close_df = df_close.iloc[n_days: , :].reset_index(drop=True)

In [16]:
new_close_df 

,Close_Date,close
0,2016-03-16,29.00
1,2016-03-17,28.98
2,2016-03-18,29.41
3,2016-03-21,28.85
4,2016-03-22,28.35
...,...,...
1476,2022-01-25,16.02
1477,2022-01-26,15.94
1478,2022-01-27,14.52
1479,2022-01-28,15.06


In [17]:
## Next, need to fill in "missing" data for n_days rows, at the end
## So, add 10 entries with same close price, to create flat line 
## Eventually on the prediction DF. 

## With the way the model trains on split data, doing this should have no
## effect on the fitting of the model, as only up to the last 10 entries will 
## be "missing", as these are the days being attempted to predict.

In [20]:
## Add n_days amount of entries to end of close_df 

## Weekday date function to try and increment Close_Date 
# from datetime import datetime

# today = datetime.today()  # get todays datetime

# if not today.weekday() == 5 or today.weekday == 6:   # if we no weekend-day

#     print(datetime.strftime(today,'%d-%m-%y')) # format it to day-month-year

last_date = new_close_df.iloc[-1]['Close_Date']
last_price = new_close_df.iloc[-1]['close']

In [23]:
last_date

Timestamp('2022-01-31 00:00:00')

In [26]:
next_date = last_date + datetime.timedelta(days=1)
next_date

Timestamp('2022-02-01 00:00:00')

In [32]:
next_date = next_date + datetime.timedelta(days=1)
next_date

Timestamp('2022-02-08 00:00:00')

In [31]:
if next_date.weekday() == 5:
    next_date = next_date + datetime.timedelta(days=2)
next_date

Timestamp('2022-02-07 00:00:00')

In [42]:
## Date thing works, will not add weekends 
## Add new rows to dataframe

## Make new empty_df
empty_df = pd.DataFrame(columns={'Close_Date','close'})
empty_df

,Close_Date,close


In [45]:
last_date = new_close_df.iloc[-1]['Close_Date']
last_price = new_close_df.iloc[-1]['close']

# new_df = pd.DataFrame(['Close_Date':last_date,last_price],columns={,'close'})
# new_df


d = {'Close_Date': [last_date], 'close': [last_price]}
df = pd.DataFrame(data=d)
df

,Close_Date,close
0,2022-01-31,16.06


In [49]:
next_date = last_date + datetime.timedelta(days=1)

d2 = {'Close_Date': [next_date], 'close': [last_price]}
df2 = pd.DataFrame(data=d2)
df2

,Close_Date,close
0,2022-02-01,16.06


In [50]:
df = df.append(df2,ignore_index=True)
df

,Close_Date,close
0,2022-01-31,16.06
1,2022-02-01,16.06


In [53]:
last_date = new_close_df.iloc[-1]['Close_Date']
last_price = new_close_df.iloc[-1]['close']

for i in range(n_days):
    if i == 0:
        next_date = last_date + datetime.timedelta(days=1)
        if next_date.weekday() == 5:
            next_date = next_date + datetime.timedelta(days=2)
        d = {'Close_Date': [last_date], 'close': [last_price]}
        df1 = pd.DataFrame(data=d)
        df1
        
    else:
        next_date = next_date + datetime.timedelta(days=1)
    
        if next_date.weekday() == 5:
            next_date = next_date + datetime.timedelta(days=2)
        
        d2 = {'Close_Date': [next_date], 'close': [last_price]}
        df2 = pd.DataFrame(data=d2)

    ## Append to df1
        df1 = df1.append(df2,ignore_index=True)

df1  ## Success 

,Close_Date,close
0,2022-01-31,16.06
1,2022-02-02,16.06
2,2022-02-03,16.06
3,2022-02-04,16.06
4,2022-02-07,16.06
5,2022-02-08,16.06
6,2022-02-09,16.06
7,2022-02-10,16.06
8,2022-02-11,16.06
9,2022-02-14,16.06


In [ ]:
## Next, join to new_close_df and prepare 'y' data

In [55]:
y_data = new_close_df.append(df1,ignore_index=True)
y_data

,Close_Date,close
0,2016-03-16,29.00
1,2016-03-17,28.98
2,2016-03-18,29.41
3,2016-03-21,28.85
4,2016-03-22,28.35
...,...,...
1486,2022-02-08,16.06
1487,2022-02-09,16.06
1488,2022-02-10,16.06
1489,2022-02-11,16.06


In [57]:
if len(y_data) == len(features_df):
    print(True)

True


In [ ]:
## Make a function which gets and resturns data, to replace prepare_data() function in mean_squared_model() below 

## Doing so in new notebook file 

In [6]:
def get_data(symbol, n_days):
    path = Path('../FilesExport_Complete_DFs_TI_noShift/'+symbol+'_TI_DF_no_shift.pkl')
    data = load_obj(path)
    df = data[symbol]
    
    df_close = df[['close']]
    df_close = df_close.reset_index().rename(columns={"Date": "Close_Date"})
    
    features_df = df.reset_index().drop(columns=['close','adjClose'])
    
    new_close_df = df_close.iloc[n_days: , :].reset_index(drop=True)
    
    ## Prevent multiple API calls each time, but use API when needed. 
    try:
        path = Path('../FilesExport_Updated_API_data/'+symbol+'_jan_2022.pkl')
        api_df = load_obj(path)
    except:
        api_df = get_FMP_historical_data(symbol)
    

    new_data = api_df[['close']]
    new_data = new_data.reset_index().rename(columns={"Date": "Close_Date"})
    new_data = new_data.iloc[0:n_days]
    
    ## Add new close data to end of close_df 
    new_close_df = new_close_df.append(new_data, ignore_index=True)
    

    
    return features_df, new_close_df

# ## Dropping QUANTITY_FAILS from dataframe before machine learning 

# def prepare_data(symbol,n_days,return_data=False):
#     features_df, new_close_df = get_data(symbol,n_days)
    
#     X = features_df.drop(columns={'Date','QUANTITY_FAILS'}).values
#     y = new_close_df['close'].values
    
#     scaler = StandardScaler().fit(X)
#     X = scaler.transform(X)
    
#     if return_data == True:
#         return X, y, features_df, new_close_df
#     else:
#         return X, y


## Rewrite above functions to fit below code 
def prepare_data(symbol,n_days):
    

In [7]:
## Takes one symbol and runs model data. 
## Call function for each individal symbol. No return data.  

def mean_squared_model(
    symbol,
    export_path,  ## Require export path to help avoiding re-writing models 
    n_days = 5, ## Default value 5, but should be tried with 1-30  
    model_count = 5, ## Number of times model runs before saving the best one. 
    validation_split_value=0.3, ## Default 0.3 for 70/30 split 
    epochs_value=400,
    units1 = 8,
    units2 = 8,
    num_of_inputs = 20,
    num_of_outputs= 1,
    model_type = 'NN'
    ):    

    X, y, = prepare_data(symbol,n_days)
    
    n_days_string = str(n_days) ## For exporting because can't concat 'int'
    accuracy_dict_symbol = str(symbol)+'_'+n_days_string
    export_path_prefix_lowAcc = export_path+'Low_Acc/'+symbol+'_'+model_type+'_'+n_days_string 
    export_path_prefix_highAcc = export_path+'High_Acc/'+symbol+'_'+model_type+'_'+n_days_string 


    for i in range(model_count):
        model_summary = {}
        ## Create Neural Network 

        # Define the model - deep neural network with two layers
        nn = Sequential()

        # First hidden layer
        nn.add(Dense(units=units1, input_dim=num_of_inputs, activation="relu"))

        # Second hidden layer
        nn.add(Dense(units=units2, activation="relu"))

        # Output layer
        nn.add(Dense(units=num_of_outputs, activation="linear"))

        # Compile the model
        nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

        # Fit the model
        model = nn.fit(X, y, 
                          validation_split=validation_split_value, 
                          epochs=epochs_value, 
                          verbose=0)
            
        ## End of if/elif

        model_loss, model_accuracy = nn.evaluate(X, y, verbose=0)
        
        model_summary = {
            'model_accuracy':model_accuracy,
            'n_days':n_days,
            'validation_split_value':validation_split_value,
            'epochs_value':epochs_value,
                        }
        
        ## Save first model 
        if i == 0:
            model_accuracy_high = model_accuracy
            model_accuracy_low = model_accuracy
            
            symbol_accuracy_dict[accuracy_dict_symbol] = {
                'model_accuracy':model_accuracy,
                'n_days':n_days
            }
            
            # Save model data Low_Acc
            nn_json = nn.to_json()
            file_path = Path(export_path_prefix_lowAcc+'_model_data.json')
            with open(file_path, "w") as json_file:
                json_file.write(nn_json)

            # Save weights
            file_path = (export_path_prefix_lowAcc+'_model_weights.h5')
            nn.save_weights(file_path)
            
            ## Save model summary 
            file_path = (export_path_prefix_lowAcc+'_model_summary.pkl')
            save_obj(model_summary,file_path)
            
            # Save model data High_Acc
            nn_json = nn.to_json()
            file_path = Path(export_path_prefix_highAcc+'_model_data.json')
            with open(file_path, "w") as json_file:
                json_file.write(nn_json)

            # Save weights
            file_path = (export_path_prefix_highAcc+'_model_weights.h5')
            nn.save_weights(file_path)
            
            ## Save model summary 
            file_path = (export_path_prefix_highAcc+'_model_summary.pkl')
            save_obj(model_summary,file_path)
            
        ## Rewrite saved model if accuracy better 
        else:
            if model_accuracy < model_accuracy_low:
                ## Rewrite values 
                model_accuracy_low = model_accuracy
                symbol_accuracy_dict[accuracy_dict_symbol] = {
                    'model_accuracy':model_accuracy,
                    'n_days':n_days
                }
                
                # Rewrite saved files
                
                ## Save model data
                nn_json = nn.to_json()
                file_path = Path(export_path_prefix_lowAcc+'_model_data.json')
                with open(file_path, "w") as json_file:
                    json_file.write(nn_json)

                # Save weights
                file_path = (export_path_prefix_lowAcc+'_model_weights.h5')
                nn.save_weights(file_path)
                
                ## Save model summary 
                file_path = (export_path_prefix_lowAcc+'_model_summary.pkl')
                save_obj(model_summary,file_path)
                
            if model_accuracy > model_accuracy_high:
                ## Rewrite values 
                model_accuracy_high = model_accuracy
                symbol_accuracy_dict[accuracy_dict_symbol] = {
                    'model_accuracy':model_accuracy,
                    'n_days':n_days
                }
                
                # Rewrite saved files
                
                ## Save model data
                nn_json = nn.to_json()
                file_path = Path(export_path_prefix_highAcc+'_model_data.json')
                with open(file_path, "w") as json_file:
                    json_file.write(nn_json)

                # Save weights
                file_path = (export_path_prefix_highAcc+'_model_weights.h5')
                nn.save_weights(file_path)
                
                ## Save model summary 
                file_path = (export_path_prefix_highAcc+'_model_summary.pkl')
                save_obj(model_summary,file_path)
    ## End of for loop 
    ## Return nothing 

In [8]:
## Import symbol list 
path = Path('../Resources/06_01_ML_symbol_success_list.pkl')
symbol_list = load_obj(path)
len(symbol_list)

770

In [9]:
# ## Create empty dict for accuracy data - only really needed when 
# ## looping multiple symbols or different models. 
# symbol_accuracy_dict = {}

# for symbol in symbol_list:
#     i = 1
#     mean_squared_model(symbol,
#                        export_path='../Model_Data/Date_Test_NN_noFTD_all/',
#                        n_days=i,
#                        epochs_value=200,
#                        validation_split_value=0.1,
#                        num_of_inputs=19
#                       )
#     i = 2
#     mean_squared_model(symbol,
#                        export_path='../Model_Data/Date_Test_NN_noFTD_all/',
#                        n_days=i,
#                        epochs_value=200,
#                        validation_split_value=0.1,
#                        num_of_inputs=19
#                       )
#     i = 5
#     mean_squared_model(symbol,
#                        export_path='../Model_Data/Date_Test_NN_noFTD_all/',
#                        n_days=i,
#                        epochs_value=200,
#                        validation_split_value=0.1,
#                        num_of_inputs=19
#                       )
#     i = 10
#     mean_squared_model(symbol,
#                        export_path='../Model_Data/Date_Test_NN_noFTD_all/',
#                        n_days=i,
#                        epochs_value=200,
#                        validation_split_value=0.1,
#                        num_of_inputs=19
#                       )

# ## Export symbol_accuracy_dict
# path = Path('../Resources/NN_noFTDall_symbol_list.pkl')
# save_obj(symbol_accuracy_dict,path)

ValueError: Data cardinality is ambiguous:
  x sizes: 149
  y sizes: 145
Make sure all arrays contain the same number of samples.

In [ ]:
symbol_accuracy_dict 

In [ ]:
## EYEG messes up when predicting for n_days = 10 
## Adding try loop to code and continuing 

for index in range(235,len(symbol_list)):
    symbol = symbol_list[index] 
    
    try:
        i = 1
        mean_squared_model(symbol,
                           export_path='../Model_Data/Date_Test_NN_noFTD_all/',
                           n_days=i,
                           epochs_value=200,
                           validation_split_value=0.1,
                           num_of_inputs=19
                          )
        i = 2
        mean_squared_model(symbol,
                           export_path='../Model_Data/Date_Test_NN_noFTD_all/',
                           n_days=i,
                           epochs_value=200,
                           validation_split_value=0.1,
                           num_of_inputs=19
                          )
        i = 5
        mean_squared_model(symbol,
                           export_path='../Model_Data/Date_Test_NN_noFTD_all/',
                           n_days=i,
                           epochs_value=200,
                           validation_split_value=0.1,
                           num_of_inputs=19
                          )
        i = 10
        mean_squared_model(symbol,
                           export_path='../Model_Data/Date_Test_NN_noFTD_all/',
                           n_days=i,
                           epochs_value=200,
                           validation_split_value=0.1,
                           num_of_inputs=19
                          )
    except: 
        continue

## Export symbol_accuracy_dict
path = Path('../Resources/NN_noFTDall_symbol_list.pkl')
save_obj(symbol_accuracy_dict,path)

In [ ]:
# ## load model data 
# file_path = Path('../Model_Data/Date_Test_NN/'+symbol+'_NN_model_data.json')
# with open(file_path, "r") as json_file:
#     model_json = json_file.read()
# loaded_model = model_from_json(model_json)

# # load weights into new model
# file_path = Path('../Model_Data/Date_Test_NN/'+symbol+'_NN_model_weights.h5')
# loaded_model.load_weights(file_path)

# # Load model summary
# file_path = Path('../Model_Data/Date_Test_NN/'+symbol+'_model_summary.pkl')
# model_summary = load_obj(file_path)

# X , y, features, close_df = prepare_data(symbol,n_days,return_data=True)


# close_df["predicted"] = loaded_model.predict(X)

In [ ]:
# export_path='../Model_Data/Date_Test_NN/'
# model_type = 'NN'

# model_summary_list = [] 

# for i in range(1,11):
#     # Load model summary
#     n_days_string = str(i)
#     file_path = Path('_model_summary.pkl')
#     model_summary = load_obj(file_path)
    
#     model_summary_list.append(model_summary)
# model_summary_list

In [ ]:
# n_days = 1
# model_type = 'NN'

# export_path='../Model_Data/Date_Test_NN_noFTD/'
# export_path_prefix = export_path+symbol+'_'+model_type+'_'+n_days_string
# n_days_string = str(n_days)

# file_path = Path(export_path_prefix+'_model_data.json')
# with open(file_path, "r") as json_file:
#     model_json = json_file.read()
# loaded_model = model_from_json(model_json)

# # load weights into new model
# file_path = Path(export_path_prefix+'_model_weights.h5')
# loaded_model.load_weights(file_path)

# X , y, features, close_df = prepare_data(symbol,n_days,return_data=True)


# close_df["predicted"] = loaded_model.predict(X)
# close_df.set_index('Close_Date',inplace=True)
# close_df[['close','predicted']].tail(30).plot(use_index=True)